<a href="https://colab.research.google.com/github/aliprg0/Google_Colab/blob/main/Binary_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install yfinance
!pip install yahooquery
from yahooquery import Screener
import yfinance as yf   
import pandas as pd
import numpy as np
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,LSTM
from tensorflow.keras.layers import Dropout
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import NearestCentroid
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
import tensorflow as tf
from sklearn.tree import DecisionTreeClassifier
from sys import getsizeof

     |████████████████████████████████| 63 kB 1.3 MB/s 
     |████████████████████████████████| 6.4 MB 11.1 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
     |████████████████████████████████| 46 kB 2.0 MB/s 


In [4]:
def read_syms_from_txt():  
  with open("syms.txt","r") as f:
    lines = f.readlines()
    f.close()
  lst = []
  for line in lines:
    line = line.strip()
    lst.append(line)
  symbols = lst
  return symbols


#symbols = ["MSFT","AAPL","GOOG","TSLA","GGPI","AI","AMZN"]
#symbols = ["BTC-USD","LTC-USD","TRX-USD","XRP-USD","ETH-USD","BNB-USD","DASH-USD","VET-USD","LINK-USD","ADA-USD","DOT-USD","SOL-USD","BCH-USD","FTT-USD","FIL-USD","XMR-USD"]
#symbols = ["AAPL","MSFT","TSLA","GOOG"]
#symbols = ["BTC-USD","ETH-USD"]
#symbols = ["BTC-USD"]

def get_crypto_syms():
   screens = [
    'all_cryptocurrencies_us']
   s = Screener()
   symbols = []
   for i in screens:
      data = s.get_screeners(i, count=250)
      dicts = data[i]['quotes']
      syms = [d['symbol'] for d in dicts]
      for sym in syms:
        symbols.append(sym)
   #print(len(symbols))
   #pieces = 15
   # new_arrays = np.array_split(symbols, pieces)
      return symbols

In [5]:
def process(dfs): 
   fixed_dfs = []
   for df in dfs:
       fixed_df = []
       df = np.array(df)
       for i in range(20, df.shape[0]-1):
           twenty_days_ago = i-20
           ninteen_days_ago = i-19
           eighteen_days_ago = i-18
           seventeen_days_ago = i-17
           sixteen_days_ago = i-16
           fifteen_days_ago = i-15
           fourteen_days_ago = i-14
           thirteen_days_ago = i-13
           twelve_days_ago = i-12
           eleven_days_ago = i-11
           ten_days_ago = i-10
           nine_days_ago = i-9
           eight_days_ago = i-8
           seven_days_ago = i-7
           six_days_ago = i-6
           five_days_ago = i-5
           four_days_ago = i-4
           three_days_ago = i-3
           two_days_ago = i - 2
           yesterday = i - 1
           today = i
           tomorrow = i + 1
           if df[tomorrow][3] > df[today][3]:
               future = 1
           else:
               future = 0


           row = [df[twenty_days_ago][3],df[ninteen_days_ago][3],df[eighteen_days_ago][3],df[seventeen_days_ago][3],
                  df[sixteen_days_ago][3],df[fifteen_days_ago][3],df[fourteen_days_ago][3],df[thirteen_days_ago][3],
                  df[twelve_days_ago][3],df[eleven_days_ago][3],df[ten_days_ago][3], df[nine_days_ago][3], 
                  df[eight_days_ago][3], df[seven_days_ago][3], df[six_days_ago][3], df[five_days_ago][3], 
                  df[four_days_ago][3], df[three_days_ago][3], df[two_days_ago][3], df[yesterday][3],
                  df[today][3], future]

           fixed_df.append(row)


       arrayed_fixed_df = np.array([fixed_df])
       two_d_fixed_df = []
       for i in range(0, arrayed_fixed_df.shape[1]):
          two_d_fixed_df.append(arrayed_fixed_df[0][i])
    
       mm = np.array(two_d_fixed_df)


       column_names = ["twenty_days_ago","ninteen_days_ago","eighteen_days_ago",
                       "seventeen_days_ago","sixteen_days_ago","fifteen_days_ago",
                       "fourteen_days_ago","thirteen_days_ago","twelve_days_ago",
                       "eleven_days_ago","ten_days_ago", "nine_days_ago", 
                       "eight_days_ago", "seven_days_ago", "six_days_ago",
                       "five_days_ago", "four_days_ago", "three_days_ago",
                       "two_days_ago", "yesterday","today", "result"]
    
       column_names_without_result = ["twenty_days_ago","ninteen_days_ago","eighteen_days_ago",
                                      "seventeen_days_ago","sixteen_days_ago","fifteen_days_ago",
                                      "fourteen_days_ago","thirteen_days_ago","twelve_days_ago",
                                      "eleven_days_ago",   "ten_days_ago", "nine_days_ago", 
                                      "eight_days_ago", "seven_days_ago", "six_days_ago",
                                      "five_days_ago", "four_days_ago", "three_days_ago", 
                                      "two_days_ago", "yesterday","today"]
       dff = pd.DataFrame(mm, columns=column_names)
       scaler = MinMaxScaler()
       result = dff["result"]
       dff = dff.drop(["result"],axis=1)
       scaled = pd.DataFrame(scaler.fit_transform(dff.T).T, columns=column_names_without_result,dtype=object)

       scaled["result"] = result
       
       fixed_dfs.append(scaled)

   data = fixed_dfs[0]
   for i in range(1,len(dfs)):
     data = pd.concat([data,fixed_dfs[i]], ignore_index = True)
   data = data.astype(float)
   data = data.dropna()
   ttext = getsizeof(data)
   # data.to_parquet(f"df_{ttext}.parquet")
   return data
   
def spliting(data):
  X = data.drop(["result"],axis=1)
  y = data["result"]
  xTrain, xTest, yTrain, yTest = train_test_split(X, y, test_size=0.2,random_state=99)
  print(xTrain.shape,end=" ")
  print(yTrain.shape)
  print(xTest.shape,end=" ")
  print(yTest.shape)
  return xTrain, xTest, yTrain, yTest

In [6]:
dfs = []
#symbols = get_crypto_syms()
symbols = read_syms_from_txt()

for i in symbols[:300]:
      data = yf.download(i,period="MAX",interval="1d",progress=False)
      if data.empty :
        print("Passing...")
      else:
          dfs.append(data)
ndfs = dfs
dfs = []
for df in ndfs:
       if df.shape[0] > 21:
          dfs.append(df)
data = process(dfs)
xTrain, xTest, yTrain, yTest = spliting(data)
data

(201877, 21) (201877,)
(22431, 21) (22431,)


,twenty_days_ago,ninteen_days_ago,eighteen_days_ago,seventeen_days_ago,sixteen_days_ago,fifteen_days_ago,fourteen_days_ago,thirteen_days_ago,twelve_days_ago,eleven_days_ago,...,eight_days_ago,seven_days_ago,six_days_ago,five_days_ago,four_days_ago,three_days_ago,two_days_ago,yesterday,today,result
0,1.000000,0.759589,0.542931,0.646041,0.572348,0.596694,0.842549,0.750563,0.665556,0.613306,...,0.401662,0.485543,0.461213,0.398775,0.285052,0.061071,0.000000,0.069937,0.114578,1.0
1,0.901536,0.644391,0.766770,0.679306,0.708200,1.000000,0.890823,0.789931,0.727917,0.685369,...,0.576279,0.547402,0.473296,0.338321,0.072484,0.000000,0.083006,0.135989,0.281313,1.0
2,0.644391,0.766770,0.679306,0.708200,1.000000,0.890823,0.789931,0.727917,0.685369,0.491590,...,0.547402,0.473296,0.338321,0.072484,0.000000,0.083006,0.135989,0.281313,0.386152,0.0
3,0.766770,0.679306,0.708200,1.000000,0.890823,0.789931,0.727917,0.685369,0.491590,0.476722,...,0.473296,0.338321,0.072484,0.000000,0.083006,0.135989,0.281313,0.386152,0.356104,1.0
4,0.679306,0.708200,1.000000,0.890823,0.789931,0.727917,0.685369,0.491590,0.476722,0.576279,...,0.338321,0.072484,0.000000,0.083006,0.135989,0.281313,0.386152,0.356104,0.362497,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
224303,0.007581,0.002624,0.059540,0.000000,0.024018,0.127725,0.119267,0.147917,0.140354,0.157160,...,0.282209,0.457954,0.323225,0.882878,0.649622,1.000000,0.866850,0.698685,0.432657,1.0
224304,0.002624,0.059540,0.000000,0.024018,0.127725,0.119267,0.147917,0.140354,0.157160,0.230742,...,0.457954,0.323225,0.882878,0.649622,1.000000,0.866850,0.698685,0.432657,0.558818,0.0
224305,0.059540,0.000000,0.024018,0.127725,0.119267,0.147917,0.140354,0.157160,0.230742,0.233746,...,0.323225,0.882878,0.649622,1.000000,0.866850,0.698685,0.432657,0.558818,0.424706,1.0
224306,0.000000,0.024018,0.127725,0.119267,0.147917,0.140354,0.157160,0.230742,0.233746,0.282209,...,0.882878,0.649622,1.000000,0.866850,0.698685,0.432657,0.558818,0.424706,0.435380,0.0


In [31]:
#for symbols in new_arrays:
#   name = f"df_{symbols[0]}.parquet"
#   dfs = []
#   for i in symbols:
#      data = yf.download(i,period="730d",interval="1h")
#      if data.empty :
#        print("Passing...")
#      else:
#          dfs.append(data)
#   ndfs = dfs
#   dfs = []
#   for df in ndfs:
#       if df.shape[0] > 21:
#          dfs.append(df)
#   process(dfs,name)

In [ ]:
clf = RandomForestClassifier(n_estimators=100)
clf.fit(xTrain,yTrain)
print(f"Random Forest :  {clf.score(xTest,yTest)}")
logisticRegr = LogisticRegression()
logisticRegr.fit(xTrain, yTrain)
print(f"Logistic Regression: {logisticRegr.score(xTest, yTest)}")
nc = NearestCentroid()
nc.fit(xTrain,yTrain)
print(f"Nearest Centroid: {nc.score(xTest,yTest)}")
gnb = GaussianNB()
gnb.fit(xTrain, yTrain)
print(f"GaussianNB: {gnb.score(xTest,yTest)}")
clf2 = KNeighborsClassifier(n_neighbors=2)
clf2.fit(xTrain, yTrain)
print(f"K-Neighbors: {clf2.score(xTest,yTest)}")
tree = DecisionTreeClassifier()
tree.fit(xTrain, yTrain)
print(f"Decision Tree: {tree.score(xTest,yTest)}")
gb = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,
   max_depth=1, random_state=0).fit(xTrain, yTrain)
gb.fit(xTrain,yTrain)
print(f"Gradient Boost: {gb.score(xTest, yTest)}")
svm = SVC()
svm.fit(xTrain,yTrain)
print(f"SVM: {svm.score(xTest,yTest)}")

Random Forest :  0.4971751412429379
Logistic Regression: 0.5819209039548022
Nearest Centroid: 0.5141242937853108
GaussianNB: 0.5310734463276836
K-Neighbors: 0.5254237288135594
Decision Tree: 0.4689265536723164
Gradient Boost: 0.536723163841808
SVM: 0.5310734463276836


In [18]:
model = Sequential()
model.add(Dense(1000, activation='relu', kernel_initializer='he_normal', input_shape=(xTrain.shape[1],)))
#model.add(Dense(1750, activation='relu', kernel_initializer='he_normal'))
#model.add(Dense(1500, activation='relu', kernel_initializer='he_normal'))
model.add(Dense(750, activation='relu', kernel_initializer='he_normal'))
model.add(Dense(1, activation='sigmoid'))


model.compile(optimizer='adam', loss='binary_crossentropy',metrics=['accuracy'])
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_27 (Dense)            (None, 1000)              22000     
                                                                 
 dense_28 (Dense)            (None, 750)               750750    
                                                                 
 dense_29 (Dense)            (None, 1)                 751       
                                                                 
Total params: 773,501
Trainable params: 773,501
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(xTrain,yTrain,epochs=15,batch_size=8,validation_data=(xTest,yTest))

Epoch 1/15
25235/25235 [==============================] - 116s 5ms/step - loss: 0.6877 - accuracy: 0.5385 - val_loss: 0.6810 - val_accuracy: 0.5537
Epoch 2/15
25235/25235 [==============================] - 107s 4ms/step - loss: 0.6779 - accuracy: 0.5590 - val_loss: 0.6830 - val_accuracy: 0.5617
Epoch 3/15
25235/25235 [==============================] - 106s 4ms/step - loss: 0.6719 - accuracy: 0.5707 - val_loss: 0.6737 - val_accuracy: 0.5740
Epoch 4/15
25235/25235 [==============================] - 111s 4ms/step - loss: 0.6664 - accuracy: 0.5815 - val_loss: 0.6707 - val_accuracy: 0.5731
Epoch 5/15
25235/25235 [==============================] - 110s 4ms/step - loss: 0.6626 - accuracy: 0.5851 - val_loss: 0.6690 - val_accuracy: 0.5713
Epoch 6/15
25235/25235 [==============================] - 111s 4ms/step - loss: 0.6582 - accuracy: 0.5922 - val_loss: 0.6661 - val_accuracy: 0.5854
Epoch 7/15
16606/25235 [==================>...........] - ETA: 34s - loss: 0.6543 - accuracy: 0.5974

In [9]:
model.evaluate(xTest,yTest)

701/701 [==============================] - 3s 5ms/step - loss: 0.6577 - accuracy: 0.6293


[0.657656729221344, 0.6292631030082703]

In [ ]:
model.save("weights_91_1500_daily_crypto.h5")